# Градиентный спуск
## Дз № 12

In [2]:
import numpy as np
import plotly.graph_objects as go

from scipy.misc import derivative

import plotly.express as px
import seaborn as sns

In [3]:
def get_der(f, h=10 ** (-12)):
    if isinstance(f, np.polynomial.Polynomial):
        return f.deriv()
    return lambda x: derivative(f, x, dx=h)

In [14]:
def gradient_descent(func, x0=0, rate=0.2, rate_decrease=0.02, eps=10 ** (-8), max_iter=100):
    x = x0
    points = [x0]
    f_der = get_der(func)
    grad = f_der(x0)
    while abs(grad) > eps:
        grad = f_der(x)
        x = x - rate * grad
        points.append(x)
        if len(points) > max_iter:
            break
        rate = (1 - rate_decrease) * rate
    return x, points

In [5]:
def f(x):
    return np.sin(x) + x ** 4 - 1.6 * x ** 3

poly = np.polynomial.Polynomial([0.6, -2, 0, 1.2, 0.5])

def g(x):
    return poly(x * 2)

In [6]:
def visualise_descent_2d(method, func, a, b, **kwargs):
    X = np.linspace(a, b, num=300)
    Y = [func(x) for x in X]
    fig = go.Figure()

    final, points = method(func, **kwargs)

    fig.add_trace(go.Scatter(x=points, y=[func(x) for x in points], mode='lines+markers', name='Путь градиента'))
    fig.add_trace(go.Scatter(x=X, y=Y, mode='lines', name='Функция f(x)'))
    fig.add_vline(x=final, name='Найденный минимум', line_dash='dot')

    fig.update_layout(autosize=False)
    fig.show()

    print(f'Num of iterations: {len(points)}')

In [15]:
visualise_descent_2d(gradient_descent, f, -0.7, 1, x0=0.4)

Num of iterations: 15


In [16]:
visualise_descent_2d(gradient_descent, g, -1.5, 1, x0=-1, rate_decrease=0.01, rate=0.2)

Num of iterations: 10


In [20]:
def nesterov_method(f, y0=0, jitter=0.1, eps=10**(-8), max_iter=100):
    x = y0
    y = y0
    a = 1
    f_der = get_der(f)
    z = y - jitter
    alpha = abs((y - z) / (f_der(y) - f_der(z)))
    points = [x]

    while abs(f_der(points[-1])) > eps:
        i = 0
        grad = f_der(y)
        while f(y) - f(y - 2 ** (-i) * alpha * grad) < 2 ** (- i - 1) * alpha * grad ** 2:
            i += 1
            if i >= 10**6:
                print(f'Jump overflow at iteration {len(points)}')
                return x, points
        alpha = 2 ** (-i) * alpha
        x = y - alpha * grad
        points.append(x)
        if len(points) > max_iter:
            break
        prev_a = a
        a = (1 + np.sqrt(4 * prev_a ** 2 + 1)) / 2
        y = x + (prev_a - 1) * (x - points[-2]) / a
    return x, points

In [21]:
visualise_descent_2d(nesterov_method, lambda x: x ** 2 + np.sin(x), -2, 2, y0=0.8)

Num of iterations: 9


In [22]:
visualise_descent_2d(nesterov_method, f, -0.7, 1, y0=0.4)

Num of iterations: 14


In [23]:
visualise_descent_2d(nesterov_method, g, -1.5, 1, y0=-1)

Num of iterations: 38
